In [2]:
# Import Dependencies
import pandas as pd
import numpy as np
import requests
import json
from appid import api_key
from pprint import pprint
from citipy import citipy
import matplotlib as plt

In [3]:
# Create lists of random longitudes and latitudes
lat = []
lng = []

for x in np.random.randint(-75,75,1200):
    lat.append(x)
    
for y in np.random.randint(-165,165,1200):
    lng.append(y)

# zip together the two lists to obtain pairs of longitude & latitudes in a tuple
lat_lng = tuple(zip(lat,lng))
#print(lat_lng)

In [4]:
# for each latitude-longitude pair we find the nearest city to that pair 
# and put the city and country in the separate lists titled city and country respectively
cities = []
countries =[]
for x, y in lat_lng:
    city = citipy.nearest_city(x, y)
    cityName = city.city_name
    cities.append(cityName)
    countries.append(city.country_code)
    
#print(cities)

In [5]:
# Creat empty lists of temperature, humidity, cloudiness and wind speed
temp = []
hum = []
cloud = []
windsp = []


In [6]:
weather_dict = {
    'City': cities,
    'Latitude': lat,
    'Longitude': lng
}

weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Latitude,Longitude
0,ustka,55,17
1,hithadhoo,-14,83
2,poum,-18,161
3,rikitea,-26,-127
4,busselton,-44,98
...,...,...,...
1195,andevoranto,-19,49
1196,saint-philippe,-44,64
1197,kamiiso,42,139
1198,souillac,-61,77


In [7]:
weather_data = weather_data.drop_duplicates(subset=["City"], keep="first")

In [8]:
# Create new columns for the other variables
weather_data['Temperature (F)'] = ""
weather_data['Humidity (%)'] = ""
weather_data['Cloudiness (%)'] = ""
weather_data['Wind Speed (mph)'] = ""

weather_data

C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ustka,55,17,,,,
1,hithadhoo,-14,83,,,,
2,poum,-18,161,,,,
3,rikitea,-26,-127,,,,
4,busselton,-44,98,,,,
...,...,...,...,...,...,...,...
1190,kegayli,45,58,,,,
1191,valparaiso,-32,-77,,,,
1195,andevoranto,-19,49,,,,
1197,kamiiso,42,139,,,,


In [20]:
# define the base url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"


# Build partial query url
query_url = f"{base_url}appid={api_key}&units={units}&q="

# Create an empty list to collect names of cities data is collected on
cityList = []

# loop through list of cities to obtain the required information for each city
for index, row in weather_data.iterrows():
    
    # get city name from dataframe
    citiName = row['City']
    
    
    # API Call
    response = requests.get(query_url + citiName).json()

    # Use 'try . . . except to skip through the cities to pull the data'
    try:
        weather_data.loc[index, 'Latitude'] = response['coord']['lat']
        weather_data.loc[index, 'Longitude'] = response['coord']['lon']
        weather_data.loc[index, 'Temperature (F)'] = response['main']['temp']
        weather_data.loc[index, 'Humidity (%)'] = response['main']['humidity']
        weather_data.loc[index, 'Cloudiness (%)'] = response['clouds']['all']
        weather_data.loc[index, 'Wind Speed (mph)'] = response['wind']['speed']
        cityList.append(citiName)
        
    except KeyError:
        weather_data.loc[index, 'Latitude'] = 'NaN'
        weather_data.loc[index, 'Longitude'] = 'NaN'
        weather_data.loc[index, 'Temperature (F)'] = 'NaN'
        weather_data.loc[index, 'Humidity (%)'] = 'NaN'
        weather_data.loc[index, 'Cloudiness (%)'] = 'NaN'
        weather_data.loc[index, 'Wind Speed (mph)'] = 'NaN'   
        print("City is missing complete weather data...skip")
        
        pass

C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yu_ka\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\U

City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is missing complete weather data...skip
City is mi

In [21]:
weather_data.head(20)

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ustka,54.5805,16.8619,3.38,74,7,3.15
1,hithadhoo,-0.6,73.0833,28.01,65,100,0.5
2,poum,-20.2333,164.017,23.54,60,0,9.62
3,rikitea,-23.1203,-134.969,25.15,70,1,4.15
4,busselton,-33.65,115.333,12.43,96,5,1.41
5,hermanus,-34.4187,19.2345,15.56,83,100,2.6
6,karratha,-20.7377,116.846,24.52,24,0,4.67
7,krasnoselkup,65.7,82.4667,0,98,100,3.48
9,tasiilaq,65.6145,-37.6368,3,75,75,1.54
10,bluff,-46.6,168.333,5,86,100,2.24


In [19]:
weather_data.count()
weather_data.dtypes

#weather_data[['Latitude','Longitude','Temperature (F)','Humidity (%)','Cloudiness (%)','Wind Speed (mph)']] = weather_data[['Latitude','Longitude','Temperature (F)','Humidity (%)','Cloudiness (%)','Wind Speed (mph)']].apply(pd.to_numeric(errors='coerce'))

#weather_data = weather_data.apply(pd.to_numeric(errors='ignore'))

#weather_data['Latitude'] = pd.to_numeric(weather_data['Latitude'])
#weather_data['Longitude'] = pd.to_numeric(weather_data['Longitude'])
#weather_data['Temperature (F)'] = pd.to_numeric(weather_data['Temperature (F)'])
#weather_data['Humidity (%)'] = pd.to_numeric(weather_data['Humidity (%)'])
#weather_data['Cloudiness (%)'] = pd.to_numeric(weather_data['Cloudiness (%)'])
#weather_data['Wind Speed (mph)'] = pd.to_numeric(weather_data['Wind Speed (mph)'])


#weather_df = pd.to_numeric(weather_data[['Latitude','Longitude','Temperature (F)','Humidity (%)','Cloudiness (%)','Wind Speed (mph)']], errors='coerce')

weather_df = weather_data.dropna(how='any')
#weather_df = weather_data[weather_data.duplicated(['City', 'Latitude', 'Longitude'], keep=False)]
weather_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ustka,54.5805,16.8619,3.38,74,7,3.15
1,hithadhoo,-0.6,73.0833,28.01,65,100,0.5
2,poum,-20.2333,164.017,23.54,60,0,9.62
3,rikitea,-23.1203,-134.969,25.15,70,1,4.15
4,busselton,-33.65,115.333,12.22,96,5,1.41
...,...,...,...,...,...,...,...
1190,kegayli,[NaN],[NaN],[NaN],[NaN],[NaN],[NaN]
1191,valparaiso,-33.0393,-71.6273,13.62,100,90,2.57
1195,andevoranto,[NaN],[NaN],[NaN],[NaN],[NaN],[NaN]
1197,kamiiso,41.8167,140.65,5.15,63,96,0.59


In [ ]:

# loop through list of cities to obtain the required information for each city
for city in cities:
    response = requests.get(query_url + city).json()
print(response)

In [14]:
print(cityList)

['ustka', 'hithadhoo', 'poum', 'rikitea', 'busselton', 'hermanus', 'karratha', 'krasnoselkup', 'tasiilaq', 'bluff', 'faya', 'port alfred', 'ennis', 'feijo', 'lumajang', 'los llanos de aridane', 'tafresh', 'takoradi', 'ancud', 'lebu', 'mulchen', 'joao pinheiro', 'punta arenas', 'luganville', 'am timan', 'namibe', 'bambous virieux', 'husavik', 'ceres', 'castro', 'airai', 'cabo san lucas', 'fort nelson', 'high point', 'albany', 'yellowknife', 'gabu', 'torbay', 'gat', 'avarua', 'latung', 'talnakh', 'carnarvon', 'padang', 'ouidah', 'arraial do cabo', 'atuona', 'bethel', 'fortuna', 'mahebourg', 'sulmona', 'sundargarh', 'havre-saint-pierre', 'saint-philippe', 'khatanga', 'kasangulu', 'hami', 'jamestown', 'hobart', 'baiyin', 'iguape', 'el sauzal', 'manjacaze', 'payson', 'broome', 'kalabo', 'raton', 'constitucion', 'saldanha', 'kieta', 'kuryk', 'pisco', 'dingli', 'petropavlovsk-kamchatskiy', 'eureka', 'zhongshu', 'ushuaia', 'mataura', 'naze', 'perene', 'saskylakh', 'marsa matruh', 'faanui', 'ta

In [16]:
# Base url for get requests
url = "https://api.openweathermap.org/data/2.5/find?"

query_url = f"{url}&lat={45}&lon={105}&cnt={2}&appid={api_key}"

response = requests.get(query_url)
weather_data = response.json()
pprint(weather_data)

{'cod': '200',
 'count': 2,
 'list': [{'clouds': {'all': 100},
           'coord': {'lat': 45.2432, 'lon': 104.8051},
           'dt': 1619471180,
           'id': 2030890,
           'main': {'feels_like': 266.15,
                    'grnd_level': 854,
                    'humidity': 57,
                    'pressure': 1023,
                    'sea_level': 1023,
                    'temp': 272.36,
                    'temp_max': 272.36,
                    'temp_min': 272.36},
           'name': 'Hashaat',
           'rain': None,
           'snow': None,
           'sys': {'country': 'MN'},
           'weather': [{'description': 'overcast clouds',
                        'icon': '04n',
                        'id': 804,
                        'main': 'Clouds'}],
           'wind': {'deg': 336, 'speed': 7.15}},
          {'clouds': {'all': 100},
           'coord': {'lat': 45.05, 'lon': 105.5667},
           'dt': 1619471181,
           'id': 2028411,
           'main': {'feels_like

In [36]:
df = pd.DataFrame(weather_data['list'])
df

,id,name,coord,main,dt,wind,sys,rain,snow,clouds,weather
0,2030890,Hashaat,"{'lat': 45.2432, 'lon': 104.8051}","{'temp': 272.96, 'feels_like': 265.33, 'temp_m...",1619399157,"{'speed': 11.78, 'deg': 0}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
1,2028411,Ulaanjirem,"{'lat': 45.05, 'lon': 105.5667}","{'temp': 275.89, 'feels_like': 269.67, 'temp_m...",1619399158,"{'speed': 10.27, 'deg': 333}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
2,2031832,Doloon,"{'lat': 44.4245, 'lon': 105.3208}","{'temp': 274.74, 'feels_like': 266.54, 'temp_m...",1619399158,"{'speed': 16.99, 'deg': 271}",{'country': 'MN'},None,None,{'all': 76},"[{'id': 803, 'main': 'Clouds', 'description': ..."
3,2028722,Tsagaan-Ovoo,"{'lat': 45.5072, 'lon': 105.7559}","{'temp': 272.6, 'feels_like': 264.71, 'temp_mi...",1619399157,"{'speed': 12.24, 'deg': 342}",{'country': 'MN'},None,{'1h': 0.312},{'all': 100},"[{'id': 600, 'main': 'Snow', 'description': 'l..."
4,2029321,Sharhulsan,"{'lat': 44.65, 'lon': 104.05}","{'temp': 276.15, 'feels_like': 270.87, 'temp_m...",1619399158,"{'speed': 7.59, 'deg': 322}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
5,2029636,Ongi,"{'lat': 45.45, 'lon': 103.9}","{'temp': 275.51, 'feels_like': 268.85, 'temp_m...",1619399157,"{'speed': 11.41, 'deg': 16}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
6,2029527,Rashaant,"{'lat': 45.3, 'lon': 106.2}","{'temp': 275.37, 'feels_like': 268.5, 'temp_mi...",1619399158,"{'speed': 12.03, 'deg': 329}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
7,2029969,Mongolia,"{'lat': 46, 'lon': 105}","{'temp': 272.9, 'feels_like': 265.28, 'temp_mi...",1619399158,"{'speed': 11.65, 'deg': 354}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
8,2029495,Sangiyn Dalay,"{'lat': 46.0079, 'lon': 104.9421}","{'temp': 273.02, 'feels_like': 265.4, 'temp_mi...",1619399157,"{'speed': 11.81, 'deg': 354}",{'country': 'MN'},None,None,{'all': 100},"[{'id': 804, 'main': 'Clouds', 'description': ..."
9,2031740,Middle Govi,"{'lat': 45.5, 'lon': 106.5}","{'temp': 273.26, 'feels_like': 265.69, 'temp_m...",1619399157,"{'speed': 11.92, 'deg': 330}",{'country': 'MN'},None,None,{'all': 74},"[{'id': 803, 'main': 'Clouds', 'description': ..."
